## Building Data Frames

**Contents**
1. Cases
2. People
3. Acts/Bills
4. Speeches
5. Keywords? (maybe add to cases)

In [140]:
import pandas as pd
import regex as re
import numpy as np
import sqlite3

In [141]:
data = pd.read_csv(r"C:\Users\Avena Cheng\Desktop\Liberating Archives\data.csv")

In [142]:
data.head()

,Unnamed: 0,Title,Case_ID,Case_No,Date,Year,Justices,Appearances,People
0,0,14 Penn Plaza LLC v. Pyett,1,No. 07-581,"December 1, 2008",2008,"['JUSTICE ALITO', 'JUSTICE BREYER', 'JUSTICE G...","PAUL SALVATORE, ESQ., New York, N.Y.; on behal...","PAUL SALVATORE, DAVID C. FREDERICK, CURTIS E. ..."
1,1,Abbott v. Perez,2,No. 17-586,"April 24, 2018",2018,"['JUSTICE ALITO', 'JUSTICE BREYER', 'JUSTICE G...","SCOTT A. KELLER, Solicitor General of Texas, ...","SCOTT A. KELLER, EDWIN S. KNEEDLER, MAX RENEA ..."
2,2,Abbott v. United States,3,No. 09-479,"October 4, 2010",2010,"['JUSTICE ALITO', 'JUSTICE BREYER', 'JUSTICE G...","DAVID L. HORAN, ESQ., Dallas, Texas; on behalf...","DAVID L. HORAN, JAMES E. RYAN"
3,3,Abdul-Kabir v. Quarterman,4,No. 05-11284,"January 17, 2006",2006,"['JUSTICE ALITO', 'JUSTICE BREYER', 'JUSTICE G...","ROBERT C. OWEN, ESQ., Austin, Tex.; on behalf...","ROBERT C. OWEN, EDWARD L. MARSHALL"
4,4,Abdur_Rahman v. Bell,5,No. 01-9094,"November 6, 2002",2002,['JUSTICE REHNQUIST'],"JAMES S. LIEBMAN, ESQ., New York, New York; on...","JAMES S. LIEBMAN, PAUL G. SUMMERS, PAUL J. ZID..."


## Cases DB

**Columns:**
* Case ID
* Title
* Case No ...notice there's some issues with this you should probably fix. 
* Date
* Year
* Month
* Day

In [143]:
data = data.drop('Unnamed: 0',axis=1)
data['Date']=pd.to_datetime(data[['Date']].stack()).unstack()

In [144]:
month = []
day = []
for i in range(len(data)):
    month += [data['Date'][i].month]
    day += [data['Date'][i].day]
data['Month'] = month
data['Day'] = day

In [145]:
case_df = data[['Title','Case_ID','Case_No','Date','Year','Month','Day']]
new_cols = [i.lower() for i in case_df.columns]

In [146]:
case_df.columns = new_cols

In [147]:
import datetime as dt

In [148]:
case_df['date'] = case_df['date'].dt.normalize()

C:\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [149]:
len(case_df)

1302

In [150]:
#case_df['date']

In [151]:
con = sqlite3.connect("database.db")

# Write the new DataFrame to a new SQLite table
case_df.to_sql("cases", con, if_exists="replace")

In [152]:
## not really useful
#roles = []
#for i in range(len(data)):
#    role = re.findall('[A-Z\s\.\,]+[A-Z][a-z]+\.',data[['Appearances']].iloc[i].values[0])
#    roles += role

In [153]:
len(set(data['Title']))

1302

## People DB

**Columns:**
* Person_ID
* Name
* Possibly role

In [154]:
data[['Appearances']].iloc[0].values[0]

'PAUL SALVATORE, ESQ., New York, N.Y.; on behalf of the Petitioners. DAVID C. FREDERICK, ESQ., Washington, D.C.; on behalf of the Respondents. CURTIS E. GANNON, ESQ., Assistant to the Solicitor General, Department of Justice, Washington, D.C.; on behalf of the United States, as amicus curiae, supporting the Respondents. '

In [155]:
data[data['Title'] == 'American Trucking Assns., Inc. v. Michigan Pub. Serv. Comm_n'][["Appearances"]].values

array([[' ROBERT DIGGES, JR., ESQ., Alexandria, Virginia; on behalf of the Petitioners . th Street, NW Suite ']],
      dtype=object)

In [156]:
# check for errors
for i in range(1302):
    lst = data['People'].iloc[i]
    if "P R O C E E D I N G S" in lst:
        print(data.iloc[i]['Title'])
    if "ARGUMENT" in lst:
        print(data.iloc[i]['Title'])
    if "JUSTCE" in lst:
        print(data.iloc[i]['Title'])
    if lst == "NW S":
        print(data.iloc[i]['Title'])

In [157]:
data['People'].iloc[0]

'PAUL SALVATORE, DAVID C. FREDERICK, CURTIS E. GANNON'

In [158]:
for i in range(1302):
    lst = data['People'].iloc[i]
    for j in lst:
        if j == "ANN O":
            print(lst)

In [159]:
everyone = []
for i in range(1302):
    lst = data['People'].iloc[i]
    lst = re.findall('[A-Z]+\s[A-Z\.\s]*[\'*A-Za-z\s*]+',data['People'].iloc[i])
    for i in lst:
        if "NW S" in i:
            continue
        if "JUSTICE" in i:
            continue
        everyone.append(i) 

In [160]:
data[['Title','People']].head()

,Title,People
0,14 Penn Plaza LLC v. Pyett,"PAUL SALVATORE, DAVID C. FREDERICK, CURTIS E. ..."
1,Abbott v. Perez,"SCOTT A. KELLER, EDWIN S. KNEEDLER, MAX RENEA ..."
2,Abbott v. United States,"DAVID L. HORAN, JAMES E. RYAN"
3,Abdul-Kabir v. Quarterman,"ROBERT C. OWEN, EDWARD L. MARSHALL"
4,Abdur_Rahman v. Bell,"JAMES S. LIEBMAN, PAUL G. SUMMERS, PAUL J. ZID..."


In [164]:
tgt = ", ".join(everyone)
print('The set being empty is a good thing. Means there are no weird names that have just a one letter last name')
set(re.findall("[A-Z]+\s[A-Z],",tgt))


The set being empty is a good thing. Means there are no weird names that have just a one letter last name


set()

In [166]:
all_justices =['GORSUCH','PHILLIPS','SCALIA','ROBERTS','KAVANAUGH','GINSBURG','BREYER','KENNEDY',
               'ALITO','STEVENS','KAGAN','REHNQUIST','THOMAS','SOTOMAYOR','SOUTER','BENNETT']

In [167]:
#re.findall('[A-Z]+\s[A-Z\.\s]*[A-Z]+',data[['People']].iloc[0][0])

In [168]:
people = all_justices+everyone
#people

### People Dataframe

#### This won't be used for the actual website. It was just helpful for cretaing the speeches dataframe.

In [172]:
people_df = pd.DataFrame({'Name':[],'Case':[]})
for i in range(1302):
    l = data['People'].iloc[i]
    title = data['Title'].iloc[i]
    l = re.findall('[A-Z]+\s[A-Z\.\s]*[\'*A-Za-z\s*]+',data['People'].iloc[i])
    for j in l:
        if "NW S" in j:
            continue
        if "JUSTICE" in j:
            continue
        people_df = people_df.append(pd.DataFrame({'Name':[j],'Case':[title]}))

In [173]:
len(people_df)

3275

In [191]:
j = {'A':0,'B':1,'C':2}

In [194]:
j.get('A')

0

In [196]:
lk = [0,0,0]

In [198]:
len(set(lk))

1

In [199]:
outputs = []

In [200]:
outputs += ['A']

In [201]:
outputs

['A']

In [174]:
people_df.head()

,Name,Case
0,PAUL SALVATORE,14 Penn Plaza LLC v. Pyett
0,DAVID C. FREDERICK,14 Penn Plaza LLC v. Pyett
0,CURTIS E. GANNON,14 Penn Plaza LLC v. Pyett
0,SCOTT A. KELLER,Abbott v. Perez
0,EDWIN S. KNEEDLER,Abbott v. Perez


In [176]:
#people_df.to_csv(r"C:\Users\Avena Cheng\Desktop\Liberating Archives\people.csv")

In [113]:
# Write the new DataFrame to a new SQLite table
#people_df.to_sql("people", con, if_exists="replace")



## Keywords/Acts&Bills Dataframe

In [181]:
keywords = pd.read_csv('frequencies.csv').drop("Unnamed: 0",axis=1)
keywords = keywords.rename({'titles':'Case'},axis=1)

In [188]:
keywords['Case'][0]

'Abuelhawa v. United States'